## IBM Applied Data Science Capstone Project Report on NYC Restaurants Search


#### Code Written By: Rohini Mandge

This model can be used by the end users who are interested in finding neighborhoods for their preferred type of cuisines. 


In [2]:
# import numpy as np # library to handle data in a vectorized manner

# import pandas as pd # library for data analsysis
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/rmandge/anaconda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libcurl-7.63.0             |       hbdb9355_0         488 KB  conda-forge
    libssh2-1.8.0              |       h5b517e9_3         232 KB  conda-forge
    pycurl-7.43.0.2            |   py36hdbc3d79_0          62 KB
    cryptography-2.3.1         |   py36hdffb7b8_0         567 KB  conda-forge
    conda-4.5.12               |        py36_1000         655 KB  conda-forge
    krb5-1.16.2                |       hbb41f41_0         1.2 MB  conda-forge
    python-3.6.6               |       h5001a0f_0        13.8 MB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    cryptography-vectors-2.3.1 |        py36_1000        30.5 MB  conda-forge
    bzip2-1.0.6                

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


**Loading the json file**

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

### Extract Required Data

All the required data for our model is in __feature__ key. This data is the list of all neighborhoods in NY city. I am going to defien a new variable which will contain this neighborhood data. 

In [6]:
neighborhoods_data = newyork_data['features']

In [7]:
type(neighborhoods_data)

list

In [8]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

### Data Transformation 
For easy data manipulation and analysis let's transform this data into pandas datafraeme.

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
#neighborhoods

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
print("Data frame created.")

Data frame created.


In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [13]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/Users/rmandge/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


**Visualize Manhattan neighborhood on the map using Folium visualiztion Librabry**

In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### Exploring the neighborhood using Foursquare

Using Foursquare API I am going to explore and segment the neighborhoods.

_Defining Foursquare Credential first_

In [15]:
CLIENT_ID = 'PUZKEXZM5W10JONOUGAPZ3VTZGZ3SZ5D1UNMHZ1J4NE3WKAN' # your Foursquare ID
CLIENT_SECRET = 'QBN3QXDJP4QP1AMWPIUO32F0YMI02IPIPMOPDUHQFDSLYJA1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PUZKEXZM5W10JONOUGAPZ3VTZGZ3SZ5D1UNMHZ1J4NE3WKAN
CLIENT_SECRET:QBN3QXDJP4QP1AMWPIUO32F0YMI02IPIPMOPDUHQFDSLYJA1


**Exploring a neighborhood in our dataframe.**

In [16]:
#manhattan_data.loc[8, 'Neighborhood']

In [19]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


**GET Request URL**

In order to fetch the information from the Foursquare API, we need to create a GET request.

In [20]:
LIMIT = 100
radius = 500
url = 'http://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url

'http://api.foursquare.com/v2/venues/explore?&client_id=PUZKEXZM5W10JONOUGAPZ3VTZGZ3SZ5D1UNMHZ1J4NE3WKAN&client_secret=QBN3QXDJP4QP1AMWPIUO32F0YMI02IPIPMOPDUHQFDSLYJA1&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [93]:
result = requests.get(url).json()["response"]['groups'][0]['items']
result

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4b4429abf964a52037f225e3-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'primary': True,
     'shortName': 'Pizza'}],
   'delivery': {'id': '72548',
    'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
      'prefix': 'https://fastly.4sqi.net/img/general/cap/',
      'sizes': [40, 50]},
     'name': 'seamless'},
    'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=72548'},
   'id': '4b4429abf964a52037f225e3',
   'location': {'address': '5198 Broadway',
    'cc': 'US',
 

In [96]:
#result['venue']['name']

#### Exploring the Neighborhoods in Manhattan.

In [97]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'VenueID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [98]:
# Calling the getNearbyVenues function to process all the Neighborhoods in Manhattan and store them
# in a data frame.
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                           latitudes=manhattan_data['Latitude'],
                           longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [126]:
print(manhattan_venues.shape)
manhattan_venues.head(50)

(3306, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,4b4429abf964a52037f225e3,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,4baf59e8f964a520a6f93be3,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.910660,4b79cc46f964a520c5122fe3,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.910660,55f81cd2498ee903149fcc64,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.910660,4b9c9c6af964a520b27236e3,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
5,Marble Hill,40.876551,-73.910660,4b5357adf964a520319827e3,Dunkin' Donuts,40.876993,-73.906507,Donut Shop
6,Marble Hill,40.876551,-73.910660,55f751ca498eacc0307d1cfe,Blink Fitness Riverdale,40.877147,-73.905837,Gym
7,Marble Hill,40.876551,-73.910660,57655be738faa66160da7527,Starbucks,40.873755,-73.908613,Coffee Shop
8,Marble Hill,40.876551,-73.910660,4a725fa1f964a520f6da1fe3,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
9,Marble Hill,40.876551,-73.910660,546d31ca498e561c698a0320,T.J. Maxx,40.877232,-73.905042,Department Store


Let's check how many venues were returned for each Venue Category.

In [100]:
manhattan_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Accessories Store,4,4,4,4,4,4,4
Adult Boutique,2,2,2,2,2,2,2
Afghan Restaurant,1,1,1,1,1,1,1
African Restaurant,3,3,3,3,3,3,3
American Restaurant,75,75,75,75,75,75,75
Animal Shelter,1,1,1,1,1,1,1
Antique Shop,3,3,3,3,3,3,3
Arcade,1,1,1,1,1,1,1
Arepa Restaurant,3,3,3,3,3,3,3


In [25]:
#print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

In [101]:
tmpManhattan = manhattan_venues.groupby('Venue Category').count()
#tmpManhattan['Venue Category'] = tmpManhattan.index
tmpManhattan.reset_index(inplace=True)
print(tmpManhattan.shape)
tmpManhattan.head()

(329, 8)


,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude
0,Accessories Store,4,4,4,4,4,4,4
1,Adult Boutique,2,2,2,2,2,2,2
2,Afghan Restaurant,1,1,1,1,1,1,1
3,African Restaurant,3,3,3,3,3,3,3
4,American Restaurant,75,75,75,75,75,75,75


### Analyze Each Venue Category

In [102]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Venue Category'] = manhattan_venues['Venue Category'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Venue Category,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,East Village,Financial District,Flatiron,Gramercy,Greenwich Village,Hamilton Heights,Hudson Yards,Inwood,Lenox Hill,Lincoln Square,Little Italy,Lower East Side,Manhattan Valley,Manhattanville,Marble Hill,Midtown,Midtown South,Morningside Heights,Murray Hill,Noho,Roosevelt Island,Soho,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
0,Pizza Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Yoga Studio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Diner,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Coffee Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Seafood Restaurant,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
manhattan_onehot.shape

(3306, 41)

**Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category.**

In [104]:
manhattan_grouped = manhattan_onehot.groupby('Venue Category').mean().reset_index()
manhattan_grouped

,Venue Category,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,East Village,Financial District,Flatiron,Gramercy,Greenwich Village,Hamilton Heights,Hudson Yards,Inwood,Lenox Hill,Lincoln Square,Little Italy,Lower East Side,Manhattan Valley,Manhattanville,Marble Hill,Midtown,Midtown South,Morningside Heights,Murray Hill,Noho,Roosevelt Island,Soho,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
0,Accessories Store,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.250000,0.000000
1,Adult Boutique,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Afghan Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,African Restaurant,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,American Restaurant,0.013333,0.013333,0.026667,0.040000,0.053333,0.026667,0.053333,0.000000,0.026667,0.026667,0.053333,0.040000,0.026667,0.000000,0.040000,0.013333,0.000000,0.026667,0.013333,0.013333,0.013333,0.013333,0.013333,0.040000,0.026667,0.040000,0.026667,0.040000,0.000000,0.026667,0.000000,0.040000,0.066667,0.013333,0.026667,0.026667,0.026667,0.013333,0.040000,0.000000
5,Animal Shelter,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Antique Shop,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Arcade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Arepa Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000
9,Argentinian Restaurant,0.000000,0.250000,0.000000,0.000000,0.00000

In [105]:
manhattan_grouped.shape

(329, 41)

__Top 5 Neighborhood for each category__

In [106]:
num_top_venues = 5

for hood in manhattan_grouped['Venue Category']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Venue Category'] == hood].T.reset_index()
    temp.columns = ['Neighborhood','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Accessories Store----
         Neighborhood  freq
0        West Village  0.25
1  Washington Heights  0.25
2     Upper West Side  0.25
3  Financial District  0.25
4   Battery Park City  0.00


----Adult Boutique----
        Neighborhood  freq
0               Noho   0.5
1       Sutton Place   0.5
2  Battery Park City   0.0
3               Soho   0.0
4        Marble Hill   0.0


----Afghan Restaurant----
        Neighborhood  freq
0         Lenox Hill   1.0
1  Battery Park City   0.0
2     Manhattanville   0.0
3            Midtown   0.0
4      Midtown South   0.0


----African Restaurant----
        Neighborhood  freq
0     Central Harlem   1.0
1  Battery Park City   0.0
2    Stuyvesant Town   0.0
3            Midtown   0.0
4      Midtown South   0.0


----American Restaurant----
  Neighborhood  freq
0      Tribeca  0.07
1    Chinatown  0.05
2      Clinton  0.05
3     Flatiron  0.05
4      Midtown  0.04


----Animal Shelter----
        Neighborhood  freq
0       Little Italy   1.0
1  


----Bus Line----
        Neighborhood  freq
0   Roosevelt Island   1.0
1  Battery Park City   0.0
2      Carnegie Hill   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Bus Station----
        Neighborhood  freq
0             Inwood  0.25
1       Hudson Yards  0.25
2   Roosevelt Island  0.25
3     Manhattanville  0.25
4  Battery Park City  0.00


----Business Service----
        Neighborhood  freq
0         Turtle Bay   1.0
1  Battery Park City   0.0
2               Soho   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Butcher----
      Neighborhood  freq
0        Yorkville  0.33
1  Lower East Side  0.33
2          Chelsea  0.33
3  Stuyvesant Town  0.00
4          Midtown  0.00


----Cafeteria----
        Neighborhood  freq
0     Central Harlem   1.0
1  Battery Park City   0.0
2    Stuyvesant Town   0.0
3            Midtown   0.0
4      Midtown South   0.0


----Café----
         Neighborhood  freq
0  Washington Heights  0.09
1      Lincoln Square  0.0

        Neighborhood  freq
0            Clinton   0.5
1            Tribeca   0.5
2  Battery Park City   0.0
3    Stuyvesant Town   0.0
4            Midtown   0.0


----Dog Run----
       Neighborhood  freq
0        Tudor City  0.18
1      Hudson Yards  0.12
2           Clinton  0.12
3  Manhattan Valley  0.06
4   Upper West Side  0.06


----Donut Shop----
         Neighborhood  freq
0  Washington Heights  0.25
1         East Harlem  0.12
2          Tudor City  0.12
3            Flatiron  0.12
4         Marble Hill  0.12


----Drugstore----
        Neighborhood  freq
0    Upper West Side   1.0
1  Battery Park City   0.0
2               Soho   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Dry Cleaner----
        Neighborhood  freq
0   Roosevelt Island   1.0
1  Battery Park City   0.0
2      Carnegie Hill   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Dumpling Restaurant----
        Neighborhood  freq
0          Chinatown  0.50
1       East Village  0.2


----Hardware Store----
        Neighborhood  freq
0       West Village   1.0
1  Battery Park City   0.0
2               Soho   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Hawaiian Restaurant----
       Neighborhood  freq
0  Manhattan Valley   0.2
1        Tudor City   0.2
2           Midtown   0.2
3       Murray Hill   0.2
4   Lower East Side   0.2


----Health & Beauty Service----
      Neighborhood  freq
0          Chelsea  0.25
1        Yorkville  0.12
2  Upper West Side  0.12
3      Murray Hill  0.12
4     Sutton Place  0.12


----Health Food Store----
       Neighborhood  freq
0  Manhattan Valley   0.2
1        Lenox Hill   0.2
2           Clinton   0.2
3        Turtle Bay   0.2
4      Hudson Yards   0.2


----Heliport----
      Neighborhood  freq
0  Stuyvesant Town   0.5
1       Tudor City   0.5
2             Soho   0.0
3      Marble Hill   0.0
4          Midtown   0.0


----High School----
        Neighborhood  freq
0     Lincoln Square   0.5
1            Tr

        Neighborhood  freq
0     Central Harlem   0.5
1   Roosevelt Island   0.5
2  Battery Park City   0.0
3    Stuyvesant Town   0.0
4            Midtown   0.0


----Mexican Restaurant----
       Neighborhood  freq
0       East Harlem  0.08
1  Hamilton Heights  0.08
2            Inwood  0.07
3        Tudor City  0.07
4      East Village  0.07


----Middle Eastern Restaurant----
      Neighborhood  freq
0  Upper West Side  0.29
1     Little Italy  0.14
2       Lenox Hill  0.14
3     East Village  0.14
4     Sutton Place  0.14


----Mini Golf----
        Neighborhood  freq
0            Tribeca   1.0
1  Battery Park City   0.0
2               Soho   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Miscellaneous Shop----
      Neighborhood  freq
0         Flatiron  0.25
1          Midtown  0.25
2     Little Italy  0.25
3             Soho  0.25
4  Stuyvesant Town  0.00


----Mobile Phone Shop----
         Neighborhood  freq
0  Washington Heights   0.8
1       Carnegie Hill 


----Plaza----
         Neighborhood  freq
0      Lincoln Square  0.29
1   Battery Park City  0.12
2          Turtle Bay  0.12
3  Financial District  0.06
4  Washington Heights  0.06


----Poke Place----
      Neighborhood  freq
0          Tribeca  0.33
1          Chelsea  0.33
2          Clinton  0.33
3  Stuyvesant Town  0.00
4          Midtown  0.00


----Pool----
         Neighborhood  freq
0           Yorkville  0.33
1            Gramercy  0.33
2  Washington Heights  0.33
3             Midtown  0.00
4       Midtown South  0.00


----Portuguese Restaurant----
          Neighborhood  freq
0             Flatiron   1.0
1       Manhattanville   0.0
2              Midtown   0.0
3        Midtown South   0.0
4  Morningside Heights   0.0


----Pub----
      Neighborhood  freq
0        Yorkville  0.11
1  Upper West Side  0.11
2      Murray Hill  0.11
3    Carnegie Hill  0.11
4         Gramercy  0.06


----Public Art----
        Neighborhood  freq
0  Battery Park City   0.5
1     Central Harl

        Neighborhood  freq
0       Sutton Place   1.0
1  Battery Park City   0.0
2               Soho   0.0
3        Marble Hill   0.0
4            Midtown   0.0


----Sporting Goods Shop----
    Neighborhood  freq
0   Civic Center  0.18
1     Lenox Hill  0.18
2       Flatiron  0.18
3        Midtown  0.18
4  Midtown South  0.06


----Sports Bar----
         Neighborhood  freq
0       Carnegie Hill   0.4
1     Upper West Side   0.2
2             Clinton   0.2
3           Yorkville   0.2
4  Washington Heights   0.0


----Sports Club----
          Neighborhood  freq
0             Flatiron   1.0
1       Manhattanville   0.0
2              Midtown   0.0
3        Midtown South   0.0
4  Morningside Heights   0.0


----Steakhouse----
         Neighborhood  freq
0             Midtown  0.13
1          Turtle Bay  0.13
2  Financial District  0.13
3             Tribeca  0.07
4         Murray Hill  0.07


----Street Art----
        Neighborhood  freq
0    Upper West Side  0.33
1        East Harlem 

----Whisky Bar----
        Neighborhood  freq
0               Noho   0.5
1            Tribeca   0.5
2  Battery Park City   0.0
3               Soho   0.0
4        Marble Hill   0.0


----Wine Bar----
      Neighborhood  freq
0     East Village  0.12
1     West Village  0.10
2       Turtle Bay  0.10
3          Tribeca  0.07
4  Upper West Side  0.07


----Wine Shop----
         Neighborhood  freq
0  Financial District  0.09
1   Battery Park City  0.07
2            Gramercy  0.07
3             Tribeca  0.07
4             Clinton  0.07


----Wings Joint----
       Neighborhood  freq
0  Manhattan Valley  0.33
1      Civic Center  0.33
2              Noho  0.33
3   Stuyvesant Town  0.00
4           Midtown  0.00


----Women's Store----
        Neighborhood  freq
0               Soho  0.26
1  Battery Park City  0.13
2           Flatiron  0.13
3       Little Italy  0.09
4            Midtown  0.09


----Yoga Studio----
       Neighborhood  freq
0          Flatiron  0.08
1     Carnegie Hill  0.0

**Creating a data frame for this data**

In [107]:
def return_most_common_nhoods(row, num_top_nhoods):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_nhoods]

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue Category']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Neighborhoods'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Neighborhoods'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Venue Category'] = manhattan_grouped['Venue Category']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_nhoods(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Venue Category,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
0,Accessories Store,West Village,Financial District,Washington Heights,Upper West Side,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
1,Adult Boutique,Sutton Place,Noho,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
2,Afghan Restaurant,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
3,African Restaurant,Central Harlem,Yorkville,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
4,American Restaurant,Tribeca,Flatiron,Chinatown,Clinton,Hudson Yards,Midtown,Gramercy,West Village,Sutton Place,Chelsea
5,Animal Shelter,Little Italy,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
6,Antique Shop,Chelsea,Civic Center,East Village,Yorkville,Gramercy,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights
7,Arcade,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
8,Arepa Restaurant,East Village,Washington Heights,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
9,Argentinian Restaurant,Lower East Side,Tribeca,East Village,Carnegie Hill,Central Harlem,Flatiron,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights


#### Cluster Cate

In [109]:
# set number of clusters
kclusters =11

manhattan_grouped_clustering = manhattan_grouped.drop('Venue Category', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]
kmeans.labels_

array([ 5, 10,  3, 10,  5,  5,  5,  2,  5,  5,  5,  5,  5,  5,  5,  6,  5,
        7,  5,  6,  9,  5,  5,  5,  5,  1,  0,  2,  5,  5,  5,  2,  5,  7,
        5,  5,  4,  6, 10, 10, 10,  4,  5,  5,  5,  5,  5,  6,  1,  5,  5,
        5, 10,  5,  5,  8, 10,  5,  1,  5,  5, 10, 10,  5, 10,  3,  5,  5,
        3,  1, 10,  8,  5,  5,  1,  5,  5,  8, 10,  5,  5,  5,  5,  3,  5,
        5,  5,  5, 10,  5,  7,  5,  5,  0,  5,  5,  5,  1,  7,  5, 10, 10,
        7,  5,  5,  5,  5,  5, 10,  5,  8,  8,  5,  5,  1,  5,  6,  5,  5,
        5,  5, 10,  5,  7,  5,  4,  4, 10,  5,  5,  5,  1,  5,  5,  5,  5,
        5,  0,  5,  5,  4,  5,  5,  5,  5,  0,  5,  5,  5,  5,  5,  5,  5,
        5,  7,  5,  5,  1,  1,  2,  8,  5,  5,  5,  4,  5,  5,  5,  5,  5,
        1,  5,  5,  9, 10,  1,  5,  5,  5,  7,  5,  5, 10,  9,  5,  6, 10,
        1,  5,  5,  0, 10,  5,  0,  9,  5,  4,  1,  5,  1, 10,  5,  5,  5,
        5,  8,  3,  5,  2,  8,  1, 10,  7,  5, 10,  1,  8,  5,  5,  5,  1,
        5,  5,  5,  5,  5

In [110]:
manhattan_merged = tmpManhattan

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Venue Category'), on='Venue Category',
                                        rsuffix='_right')

manhattan_merged.head(10)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,VenueID,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
0,Accessories Store,4,4,4,4,4,4,4,5,West Village,Financial District,Washington Heights,Upper West Side,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
1,Adult Boutique,2,2,2,2,2,2,2,10,Sutton Place,Noho,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
2,Afghan Restaurant,1,1,1,1,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
3,African Restaurant,3,3,3,3,3,3,3,10,Central Harlem,Yorkville,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
4,American Restaurant,75,75,75,75,75,75,75,5,Tribeca,Flatiron,Chinatown,Clinton,Hudson Yards,Midtown,Gramercy,West Village,Sutton Place,Chelsea
5,Animal Shelter,1,1,1,1,1,1,1,5,Little Italy,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
6,Antique Shop,3,3,3,3,3,3,3,5,Chelsea,Civic Center,East Village,Yorkville,Gramercy,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights
7,Arcade,1,1,1,1,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
8,Arepa Restaurant,3,3,3,3,3,3,3,5,East Village,Washington Heights,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
9,Argentinian Restaurant,4,4,4,4,4,4,4,5,Lower East Side,Tribeca,East Village,Carnegie Hill,Central Harlem,Flatiron,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights


In [111]:
manhattan_merged.shape

(329, 19)

In [112]:
# # create map
# map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# # set color scheme for the clusters
# x = np.arange(kclusters)
# ys = [i+x+(i*x)**2 for i in range(kclusters)]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# # add markers to the map
# markers_colors = []
# for lat, lon, poi, cluster in zip(manhattan_merged['Neighborhood Latitude'], manhattan_merged['Neighborhood Longitude'], manhattan_merged['Venue Category'], manhattan_merged['Cluster Labels']):
#     label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
#     folium.CircleMarker(
#         [lat, lon],
#         radius=3,
#         popup=label,
#         color=rainbow[cluster-1],
#         fill=True,
#         fill_color=rainbow[cluster-1],
#         fill_opacity=0.7).add_to(map_clusters)
       
# map_clusters

In [113]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
26,Basketball Court,2,2,2,0,Tribeca,Stuyvesant Town,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
93,Dive Bar,2,2,2,0,Tribeca,Clinton,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
137,Gym Pool,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
145,High School,2,2,2,0,Lincoln Square,Tribeca,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
190,Mini Golf,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
193,Modern European Restaurant,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
229,Pier,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
265,Skate Park,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
319,Volleyball Court,1,1,1,0,Tribeca,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
323,Whisky Bar,2,2,2,0,Tribeca,Noho,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy


In [114]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
25,Baseball Field,2,2,2,1,Roosevelt Island,Stuyvesant Town,Yorkville,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
48,Bus Line,1,1,1,1,Roosevelt Island,Yorkville,Little Italy,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
58,Caucasian Restaurant,2,2,2,1,Hudson Yards,Clinton,Yorkville,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hamilton Heights,Greenwich Village,Gramercy
69,College Bookstore,1,1,1,1,Lincoln Square,Yorkville,West Village,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
74,Concert Hall,8,8,8,1,Lincoln Square,Midtown,Hudson Yards,Clinton,Flatiron,Lenox Hill,Inwood,Hamilton Heights,Greenwich Village,Gramercy
97,Dry Cleaner,1,1,1,1,Roosevelt Island,Yorkville,Little Italy,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
114,Food Court,6,6,6,1,Midtown South,Battery Park City,Clinton,Manhattanville,Financial District,Chinatown,Civic Center,Chelsea,East Harlem,East Village
131,Golf Course,1,1,1,1,Midtown South,Yorkville,Little Italy,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
157,Indie Movie Theater,7,7,7,1,Lincoln Square,Noho,Chinatown,Tribeca,Greenwich Village,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Gramercy
158,Indie Theater,7,7,7,1,Clinton,Chelsea,Tribeca,Midtown South,Lincoln Square,Roosevelt Island,Yorkville,Flatiron,Inwood,Hudson Yards


In [115]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
7,Arcade,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
27,Bed & Breakfast,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
31,Belgian Restaurant,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
159,Irish Pub,2,2,2,2,Gramercy,Sutton Place,Yorkville,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
208,North Indian Restaurant,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
270,Social Club,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
302,Thrift / Vintage Store,5,5,5,2,Gramercy,West Village,Yorkville,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron
304,Tiki Bar,1,1,1,2,Gramercy,Yorkville,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Flatiron


In [116]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
2,Afghan Restaurant,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
65,Club House,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
68,College Academic Building,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
83,Czech Restaurant,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
206,Non-Profit,1,1,1,3,Lenox Hill,Yorkville,West Village,Lincoln Square,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron


In [117]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
36,Board Shop,1,1,1,4,West Village,Yorkville,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
41,Brazilian Restaurant,2,2,2,4,West Village,Midtown,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
125,Gastropub,10,10,10,4,West Village,Midtown South,Manhattan Valley,Hamilton Heights,Tribeca,Murray Hill,Yorkville,Flatiron,Hudson Yards,Greenwich Village
126,Gay Bar,3,3,3,4,West Village,Carnegie Hill,Yorkville,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village
140,Hardware Store,1,1,1,4,West Village,Yorkville,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
164,Jazz Club,9,9,9,4,West Village,Upper East Side,Greenwich Village,Murray Hill,East Village,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton
196,Moroccan Restaurant,2,2,2,4,East Village,West Village,Yorkville,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village
227,Piano Bar,1,1,1,4,West Village,Yorkville,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron


In [119]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 6, 
                     manhattan_merged.columns[[0] + list(range(5, manhattan_merged.shape[1]))]]

,Venue Category,Venue,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Neighborhoods,2nd Most Common Neighborhoods,3rd Most Common Neighborhoods,4th Most Common Neighborhoods,5th Most Common Neighborhoods,6th Most Common Neighborhoods,7th Most Common Neighborhoods,8th Most Common Neighborhoods,9th Most Common Neighborhoods,10th Most Common Neighborhoods
15,Auditorium,1,1,1,6,Battery Park City,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
19,BBQ Joint,6,6,6,6,Battery Park City,Manhattanville,Central Harlem,Lenox Hill,Greenwich Village,Flatiron,Inwood,Hudson Yards,Hamilton Heights,Gramercy
37,Boat or Ferry,4,4,4,6,Battery Park City,Hudson Yards,Stuyvesant Town,Financial District,Lenox Hill,Inwood,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
47,Burrito Place,2,2,2,6,Battery Park City,Upper East Side,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
116,Fountain,3,3,3,6,Battery Park City,Lincoln Square,West Village,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
185,Memorial Site,2,2,2,6,Battery Park City,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
238,Public Art,2,2,2,6,Battery Park City,Central Harlem,Flatiron,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy
258,Sculpture Garden,2,2,2,6,Battery Park City,Upper East Side,Financial District,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
264,Shopping Mall,3,3,3,6,Battery Park City,Marble Hill,Little Italy,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron
309,Tree,1,1,1,6,Battery Park City,West Village,Lincoln Square,Lenox Hill,Inwood,Hudson Yards,Hamilton Heights,Greenwich Village,Gramercy,Flatiron


In [127]:
venue_id = '4b5357adf964a520319827e3' # Seafood Restaurant
urlSeaFood = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
urlSeaFood

'https://api.foursquare.com/v2/venues/4b5357adf964a520319827e3?client_id=PUZKEXZM5W10JONOUGAPZ3VTZGZ3SZ5D1UNMHZ1J4NE3WKAN&client_secret=QBN3QXDJP4QP1AMWPIUO32F0YMI02IPIPMOPDUHQFDSLYJA1&v=20180605'

In [131]:
resultSeaFood = requests.get(urlSeaFood).json()
print(resultSeaFood['response']['venue'].keys())
resultSeaFood['response']['venue']
#result['response']['venue']['rating']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards',
      'displayValue': 'Yes (incl. Discover & Visa)'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'No'}],
    'name': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 1,
    'items': [{'displayName': 'Wi-Fi', 'displayValue': 'No'}],
    'name': 'Wi-Fi',
    'type': 'wifi'},
   {'count': 8,
    'items': [{'displayName': 'Breakfast', 'displayValue': 'Breakfast'}],
    'name': 'Menus',
    'summary': 'Breakfast',
    'type': 'serves'},
   {'count': 5,
    'items': [{'displayName': 'Delivery', 'displayValue': 'No Delivery'}],
    'name': 'Dining Options',
    'type': 'diningOptions'}]},
 'be

In [140]:
# try:
#     print(resultSeaFood['response']['venue']['rating'])
# except:
#     print('This venue has not been rated yet.')

In [139]:
# resultSeaFood['response']['venue']['tips']['count']

In [141]:
#Finding Italian restaurant
address = 'Manhattan, NY'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/Users/rmandge/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


40.7900869 -73.9598295


In [228]:
search_query = 'Italian'
radius = 5000

In [229]:
urlItalian = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
urlItalian

'https://api.foursquare.com/v2/venues/search?client_id=PUZKEXZM5W10JONOUGAPZ3VTZGZ3SZ5D1UNMHZ1J4NE3WKAN&client_secret=QBN3QXDJP4QP1AMWPIUO32F0YMI02IPIPMOPDUHQFDSLYJA1&ll=40.7900869,-73.9598295&v=20180605&query=Italian&radius=5000&limit=100'

In [230]:
results_italian = requests.get(urlItalian).json()
results_italian

{'meta': {'code': 200, 'requestId': '5c1fe2624c1f677d70beff7c'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'delivery': {'id': '294727',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/carmines-upper-west-side-2450-broadway--new-york/294727?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294727'},
    'hasPerk': False,
    'id': '4a7778a1f964a5209be41fe3',
    'location': {'address': '2450 Broadway',
     'cc': 'US',
     'city': 'New York',
     'country': 'United State

In [239]:
venues_IT = results_italian['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues_IT)
print(dataframe.columns)
#dataframe.head()

Index(['categories', 'delivery.id', 'delivery.provider.icon.name',
       'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes',
       'delivery.provider.name', 'delivery.url', 'hasPerk', 'id',
       'location.address', 'location.cc', 'location.city', 'location.country',
       'location.crossStreet', 'location.distance',
       'location.formattedAddress', 'location.labeledLatLngs', 'location.lat',
       'location.lng', 'location.neighborhood', 'location.postalCode',
       'location.state', 'name', 'referralId', 'venuePage.id'],
      dtype='object')


In [243]:
# Keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith(
    'location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# # function that extracts the category of the venue
# def get_nhood_type(row):
#     try:
#         nhood_list = row['location.neighborhood']
#     except:
#         nhood_list = row['location.neighborhood']
    
#     #print("nhood_list:",nhood_list)
#     return nhood_list
# #     if len(nhood_list) == 0:
# #         return None
# #     else:
# #         return nhood_list[0]['name']

# # filter the nhood for each row
# dataframe_filtered['neighborhood'] = dataframe_filtered.apply(get_nhood_type, axis=1)




# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Carmine's Italian Restaurant,Italian Restaurant,2450 Broadway,US,New York,United States,btwn W 90th & W 91st,1198,"[2450 Broadway (btwn W 90th & W 91st), New York, NY 10024, United States]","[{'label': 'display', 'lat': 40.7910963, 'lng': -73.9739914}]",40.791096,-73.973991,NaN,10024,NY,4a7778a1f964a5209be41fe3
1,Quality Italian,Italian Restaurant,57 W 57th St,US,New York,United States,at 6th Ave,3198,"[57 W 57th St (at 6th Ave), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.76447598357648, 'lng': -73.97703230787566}]",40.764476,-73.977032,NaN,10019,NY,51e7310c498e639ed27062b1
2,Carmine’s Italian Restaurant,Italian Restaurant,200 W 44th St,US,New York,United States,btwn Broadway & 8th Ave,4280,"[200 W 44th St (btwn Broadway & 8th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.7574973, 'lng': -73.9867788}]",40.757497,-73.986779,NaN,10036,NY,3fd66200f964a5209ee81ee3
3,The Italian Academy (Casa Italiana),General College & University,1161 Amsterdam Ave,US,New York,United States,West 118th Street,1955,"[1161 Amsterdam Ave (West 118th Street), New York, NY 10027, United States]","[{'label': 'display', 'lat': 40.80764460477974, 'lng': -73.96039587567955}]",40.807645,-73.960396,NaN,10027,NY,4ba00318f964a520285237e3
4,Bellini Italian Restaurant & Brick Oven Pizza,Italian Restaurant,483 Columbus Ave,US,New York,United States,btwn 83rd & 84th St,1302,"[483 Columbus Ave (btwn 83rd & 84th St), New York, NY 10024, United States]","[{'label': 'display', 'lat': 40.784655556336745, 'lng': -73.97352196636533}]",40.784656,-73.973522,NaN,10024,NY,4b786cecf964a52052cd2ee3
5,Italian Village Pizzeria,Pizza Place,1526 1st Ave,US,New York,United States,79th and 1st,2039,"[1526 1st Ave (79th and 1st), New York, NY 10075, United States]","[{'label': 'display', 'lat': 40.772669, 'lng': -73.952319}]",40.772669,-73.952319,Yorkville,10075,NY,4a90dbbef964a520a11920e3
6,Harry's Italian Pizza Bar,Pizza Place,30 Rockefeller Plz,US,New York,United States,NaN,3803,"[30 Rockefeller Plz, New York, NY 10112, United States]","[{'label': 'display', 'lat': 40.759155215362796, 'lng': -73.979006510208}]",40.759155,-73.979007,NaN,10112,NY,4e3736cab61c5cb3c37e529e
7,Mama Mia 44sw Italian Cuisine,Italian Restaurant,621 9th Ave,US,New York,United States,NaN,4325,"[621 9th Ave, New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75971578462592, 'lng': -73.9918286373273}]",40.759716,-73.991829,NaN,10036,NY,4f44a2bd19836ed0019571be
8,Rose & Joe's Italian Bakery,Bakery,22-40 31st St,US,Astoria,United States,btwn Ditmars & 23rd Ave.,4456,"[22-40 31st St (btwn Ditmars & 23rd Ave.), Astoria, NY 11105, United States]","[{'label': 'display', 'lat': 40.776943, 'lng': -73.909889}]",40.776943,-73.909889,NaN,11105,NY,4b8953b2f964a520e32b32e3
9,Davio's North Italian Steakhouse,Italian Restaurant,447 Lexington Ave,US,New York,United States,45th Street,4291,"[447 Lexington Ave (45th Street), New York, NY 10017, United States]","[{'label': 'display', 'lat': 40.753233339315564, 'lng': -73.97477368607031}]",40.753233,-73.974774,NaN,10017,NY,525d7bde498e3a406f590b1d


In [244]:
dataframe_filtered.dtypes

name                object 
categories          object 
address             object 
cc                  object 
city                object 
country             object 
crossStreet         object 
distance            int64  
formattedAddress    object 
labeledLatLngs      object 
lat                 float64
lng                 float64
neighborhood        object 
postalCode          object 
state               object 
id                  object 
dtype: object

In [246]:
searchedDF = dataframe_filtered[['name','categories','address','crossStreet','neighborhood']].copy()
searchedDF.columns=['Name','Category','Address','Cross Street','Neighborhood']
searchedDF

,Name,Category,Address,Cross Street,Neighborhood
0,Carmine's Italian Restaurant,Italian Restaurant,2450 Broadway,btwn W 90th & W 91st,NaN
1,Quality Italian,Italian Restaurant,57 W 57th St,at 6th Ave,NaN
2,Carmine’s Italian Restaurant,Italian Restaurant,200 W 44th St,btwn Broadway & 8th Ave,NaN
3,The Italian Academy (Casa Italiana),General College & University,1161 Amsterdam Ave,West 118th Street,NaN
4,Bellini Italian Restaurant & Brick Oven Pizza,Italian Restaurant,483 Columbus Ave,btwn 83rd & 84th St,NaN
5,Italian Village Pizzeria,Pizza Place,1526 1st Ave,79th and 1st,Yorkville
6,Harry's Italian Pizza Bar,Pizza Place,30 Rockefeller Plz,NaN,NaN
7,Mama Mia 44sw Italian Cuisine,Italian Restaurant,621 9th Ave,NaN,NaN
8,Rose & Joe's Italian Bakery,Bakery,22-40 31st St,btwn Ditmars & 23rd Ave.,NaN
9,Davio's North Italian Steakhouse,Italian Restaurant,447 Lexington Ave,45th Street,NaN


**Resulting Table**

Above table show all the restaurants information for which the search was performed. In this case all the Italian Restaurants.

In [234]:
## Showing these records on map
from IPython.display import HTML, display

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe['location.lat'], dataframe['location.lng'], dataframe_filtered.name):
    #print("Label--------",label)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=folium.Popup(label, parse_html=True),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# add a red circle marker to represent neighborhood
for lat, lng, label in zip(dataframe['location.lat'], dataframe['location.lng'], dataframe_filtered.neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=folium.Popup(label, parse_html=True),
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map
venues_map

**Resulting map**

The above map shows all the restaurants for 'Italian Cuisine' in blue and Neighborhoods in red. In the data extracted from folium API, some neighborhood information is not available. Map is showing only those records which contained the neighborhood names.

**Following code finds out the review text for a particular restaurant**

In [235]:
limit = 25 # set limit to be greater than or equal to the total number of tips
venue_id = '51e7310c498e639ed27062b1' # Carmine's Italian Restaurant
urlTmp = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

resultsTmp = requests.get(urlTmp).json()
resultsTmp

{'meta': {'code': 200, 'requestId': '5c1fe263351e3d5f52fd9985'},
 'response': {'tips': {'count': 186,
   'items': [{'agreeCount': 2,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/56dd7dbf498e359a31b28e1d',
     'createdAt': 1457356223,
     'disagreeCount': 0,
     'id': '56dd7dbf498e359a31b28e1d',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'photo': {'createdAt': 1457356224,
      'height': 1440,
      'id': '56dd7dc0498e666330d37a7a',
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'suffix': '/7835045_FMebIJu0zJ0kg3FXh-b145edqSCFqXDncB_wW9vneRw.jpg',
      'visibility': 'public',
      'width': 1920},
     'photourl': 'https://fastly.4sqi.net/img/general/original/7835045_FMebIJu0zJ0kg3FXh-b145edqSCFqXDncB_wW9vneRw.jpg',
     'text': "Everything was outstanding! The service, the co

In [236]:
tips = resultsTmp['response']['tips']['items']
tip = resultsTmp['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [237]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
print(tips_filtered.T)

                                                                                                                                                                                                                      0
text            Everything was outstanding! The service, the cocktails, the food! Don't miss the corn soufflé, the parm pizza or the raw bar! Next time we will definitely bring more people so we can try more dishes!
agreeCount      2                                                                                                                                                                                                      
disagreeCount   0                                                                                                                                                                                                      
id              56dd7dbf498e359a31b28e1d                                                                                                